In [1]:
import pandas as pd
import numpy as np

In [5]:
# understanding stem != original form
stemmer.stem('riddled')

u'riddl'

In [14]:
# filtering the non-SVO out
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
with open('svo.txt') as f:
    results = []
    lines = f.readlines()
    for line in lines:
        line = [token.strip() for token in line.split(',') if token.strip()]
        initials = [each[0] for each in line if each]
        initials = set(''.join(initials))
        if initials == set('SVO'):
            results.append(line)
    with open('final_svo.txt', 'w') as out:
#       line: list of tuple pairs 
        for line in results:
            line_dict = {}
            for i, pair in enumerate(line):
                pair = pair.split(':')
                if len(pair) == 2:
                    label = pair[0].strip()
                    word = pair[1].strip().strip('""').lower()
                    if label == 'V':
                        word = stemmer.stem(word)
#                         print('%s: %s' % (label, word))
                    line_dict.update({label: word})
            out.write('%s: %-15s, ' % ('S', line_dict['S']))
            out.write('%s: %-15s, ' % ('V', line_dict['V']))
            out.write('%s: %-15s' % ('O', line_dict['O']))
            out.write('\n')

In [15]:
with open('final_svo.txt') as f:
    lines = f.readlines()
    print(len(lines))

638


In [16]:
# filtering SVO that is beyond our purpose
verb_df = pd.read_csv('verbs.csv')
print(verb_df.columns)
verb_df.head()

Index([u'Verb', u'Frequency Verb', u'WordNet', u'PC-ACE Verbal Phrase',
       u'Student check (if social action put an X in the appropriate cell)',
       u'Confirmed social verb'],
      dtype='object')


,Verb,Frequency Verb,WordNet,PC-ACE Verbal Phrase,Student check (if social action put an X in the appropriate cell),Confirmed social verb
0,lynch,890.0,x,x,x,x
1,read,30.0,x,NaN,x,x
2,tell,662.0,x,x,x,x
3,drive,189.0,x,x,x,x
4,take,2022.0,x,x,x,x


In [17]:
confirmed = verb_df['Confirmed social verb'].isnull()

In [18]:
idx = verb_df[confirmed==False].index.tolist()

In [19]:
verbs = set(verb_df.loc[idx]['Verb'].tolist())

In [30]:
actors_df = pd.read_csv('actors.csv')
confirmed = actors_df['Confirmed social noun'].isnull()
idx = actors_df[confirmed == False].index.tolist()
# print(actors_df.loc[idx].head())
actors = actors_df.loc[idx]['Stanford Noun']
actors = set(actor.lower() for actor in actors)

In [31]:
print(actors_df.head())

  Stanford Noun  Frequency Noun WordNet Social Actor  \
0           man            3681                    x   
1         negro            7083                    x   
2           mob            2171                    x   
3        whites             123                  NaN   
4        parent              19                    x   

  PC-ACE Individual or Collective Actor  \
0                                     x   
1                                     x   
2                                     x   
3                                     x   
4                                   NaN   

  Student check (if social actor put an X in the appropriate cell)  \
0                                                  x                 
1                                                  x                 
2                                                  x                 
3                                                  x                 
4                                                  x  

In [32]:

with open('final_svo.txt','r') as f:
    final_svo = []
    lines = f.readlines()
    for l in lines:
        isConfirmed = True
        for pair in l.strip().split(','):
#             print(pair)
            if len(pair.split(':'))==2:
                label, word = pair.split(':')
            else:
                isConfirmed = False
                break
            label = label.strip()
            word = word.strip()
#           obtain the stem of word
            if label == 'S' and word not in actors:
                isConfirmed = False
                break
#             if label =='V' and word not in verbs:
#                 isConfirmed = False
#                 break
        if isConfirmed:
            final_svo.append(l)
#         print(final_svo)
    with open('terminal_svo.txt', 'w') as out:
        print('There are %d number of SVOs extracted from 100 lynching articles.' % len(final_svo))
        for line in final_svo:
            out.write(line)
#             out.write('\n')

There are 184 number of SVOs extracted from 100 lynching articles.
